# Solicitud Credito Main

In [1]:
import numpy as np
import pandas as pd
import sqlite3
from sqlalchemy import create_engine,text
from datetime import datetime


### Metadata

In [2]:
dir_server = 'production-finky.ceyqdhdvvqo1.us-east-1.rds.amazonaws.com'
nDB = 'finky'
nUser = 'data-finky'
vPass = 'Finky2023***'
vpuerto = '5432'

nDWH = 'dwh_finky.db'

strCnn = f'postgresql+psycopg2://{nUser}:{vPass}@{dir_server}:{vpuerto}/{nDB}'

## Dimension

#### Universidades

In [3]:
vsql="""

select id, name, contact_full_name, contact_phone, contact_email from universities

"""

try:
   
    engine = create_engine(strCnn)
    cnn = engine.connect()
    result = cnn.execute(text(vsql))
    
    df_duni=pd.DataFrame(result.fetchall(), columns=result.keys())
  
    
except Exception as e:
    print("Ocurrío un error la intentar conectarse con el servidor de BD: ", e)

finally:
    cnn.close()

print('cantidad de registros {}'.format(df_duni.shape[0]))


df_duni.head() 

cantidad de registros 59


,id,name,contact_full_name,contact_phone,contact_email
0,53,Universidad de América,Fabian Zambrano,6016580658,planeacion.financiera@uamerica.edu.co
1,57,Educación Colombo Japonesa,Elizabeth Ortiz Alarcon,3160232388,gerente.admon@colombojaponesa.com
2,59,Incad,Olga Lucia Forero,3158953531,olga.forero@incad.edu.co
3,54,Universidad De América - Diplomados,Fabian Zambrano,6016580658,planeacion.financiera@uamerica.edu.co
4,58,English4 - 180 Horas,Carol,3124352462,carol@english4.co


In [4]:
# Separando universidad del programa academico

dfx =df_duni.name.str.split('-',expand=True)[[0,1]]
dfx.columns=['Institución','Programa']
df_duni = pd.concat([df_duni,dfx],axis=1)
df_duni.head()

,id,name,contact_full_name,contact_phone,contact_email,Institución,Programa
0,53,Universidad de América,Fabian Zambrano,6016580658,planeacion.financiera@uamerica.edu.co,Universidad de América,None
1,57,Educación Colombo Japonesa,Elizabeth Ortiz Alarcon,3160232388,gerente.admon@colombojaponesa.com,Educación Colombo Japonesa,None
2,59,Incad,Olga Lucia Forero,3158953531,olga.forero@incad.edu.co,Incad,None
3,54,Universidad De América - Diplomados,Fabian Zambrano,6016580658,planeacion.financiera@uamerica.edu.co,Universidad De América,Diplomados
4,58,English4 - 180 Horas,Carol,3124352462,carol@english4.co,English4,180 Horas


In [5]:
conn = sqlite3.connect(nDWH)
cursor = conn.cursor()

# Crear la tabla dimUniversidad si no existe
cursor.execute('''
CREATE TABLE IF NOT EXISTS dimUniversidad (
    id INTEGER PRIMARY KEY,
    name TEXT,
    contact_full_name TEXT,
    contact_phone TEXT,
    contact_email TEXT,
    Institución TEXT,
    Programa TEXT
)
''')

# Asegurarse de que todos los tipos de datos sean compatibles con SQLite (por ejemplo, convertir a string)
df_duni['id'] = df_duni['id'].astype(int)
df_duni['name'] = df_duni['name'].astype(str)
df_duni['contact_full_name'] = df_duni['contact_full_name'].astype(str)
df_duni['contact_phone'] = df_duni['contact_phone'].astype(str)
df_duni['contact_email'] = df_duni['contact_email'].astype(str)
df_duni['Institución'] = df_duni['Institución'].astype(str)
df_duni['Programa'] = df_duni['Programa'].astype(str)

# Insertar los datos del DataFrame en la tabla dimUniversidad
for _, row in df_duni.iterrows():
    cursor.execute('''
    INSERT INTO dimUniversidad (id, name, contact_full_name, contact_phone, contact_email, Institución, Programa)
    VALUES (?, ?, ?, ?, ?, ?, ?)
    ''', (row['id'], row['name'], row['contact_full_name'], row['contact_phone'], row['contact_email'], row['Institución'], row['Programa']))

# Guardar los cambios y cerrar la conexión
conn.commit()
conn.close()

#### Tiempo

In [128]:
def crear_dimension_tiempo(fecha_inicio, fecha_fin):
    # Generar un rango de fechas
    fechas = pd.date_range(start=fecha_inicio, end=fecha_fin, freq='D')
    
    # Crear un DataFrame de la dimensión de tiempo
    df_dtiempo = pd.DataFrame(fechas, columns=['Fecha'])
    
    # Usar el índice como ID
    df_dtiempo['ID'] = df_dtiempo.index + 1
    
    # Agregar columnas adicionales
    df_dtiempo['Año'] = df_dtiempo['Fecha'].dt.year
    df_dtiempo['Mes'] = df_dtiempo['Fecha'].dt.month
    df_dtiempo['Día'] = df_dtiempo['Fecha'].dt.day
    df_dtiempo['Día_Semana'] = df_dtiempo['Fecha'].dt.dayofweek
    df_dtiempo['Nombre_Día_Semana'] = df_dtiempo['Fecha'].dt.day_name()
    df_dtiempo['Trimestre'] = df_dtiempo['Fecha'].dt.quarter
    df_dtiempo['Semana_Año'] = df_dtiempo['Fecha'].dt.isocalendar().week
    df_dtiempo['Es_Fin_de_Semana'] = df_dtiempo['Día_Semana'].apply(lambda x: 1 if x >= 5 else 0)
    
    # Agregar una fila para manejar fechas nulas
    fecha_nula = datetime(1900, 1, 1)
    fila_nula = pd.DataFrame({
        'Fecha': [fecha_nula],
        'ID': [0],  # ID para la fecha nula
        'Año': [fecha_nula.year],
        'Mes': [fecha_nula.month],
        'Día': [fecha_nula.day],
        'Día_Semana': [fecha_nula.weekday()],
        'Nombre_Día_Semana': [fecha_nula.strftime('%A')],
        'Trimestre': [(fecha_nula.month - 1) // 3 + 1],
        'Semana_Año': [fecha_nula.isocalendar()[1]],
        'Es_Fin_de_Semana': [1 if fecha_nula.weekday() >= 5 else 0]
    })
    
    # Concatenar la fila nula al DataFrame
    df_dtiempo = pd.concat([fila_nula, df_dtiempo], ignore_index=True)
    
    return df_dtiempo

# Definir el rango de fechas automáticamente
fecha_inicio = '2020-01-01'
fecha_fin = datetime(datetime.now().year, 12, 31)

# Crear la dimensión de tiempo
df_dtiempo = crear_dimension_tiempo(fecha_inicio, fecha_fin)

# Mostrar los primeros registros de la dimensión de tiempo
df_dtiempo.head()

,Fecha,ID,Año,Mes,Día,Día_Semana,Nombre_Día_Semana,Trimestre,Semana_Año,Es_Fin_de_Semana
0,1900-01-01,0,1900,1,1,0,Monday,1,1,0
1,2020-01-01,1,2020,1,1,2,Wednesday,1,1,0
2,2020-01-02,2,2020,1,2,3,Thursday,1,1,0
3,2020-01-03,3,2020,1,3,4,Friday,1,1,0
4,2020-01-04,4,2020,1,4,5,Saturday,1,1,1


In [130]:
def guardar_dimension_tiempo_sqlite(df, db_name, table_name):
    # Conectar a SQLite (o crear la base de datos si no existe)
    conexion = sqlite3.connect(db_name)
    cursor = conexion.cursor()
    
    # Crear la tabla si no existe
    cursor.execute(f"""
    CREATE TABLE IF NOT EXISTS {table_name} (
        Fecha TEXT,
        ID INTEGER PRIMARY KEY,
        Año INTEGER,
        Mes INTEGER,
        Día INTEGER,
        Día_Semana INTEGER,
        Nombre_Día_Semana TEXT,
        Trimestre INTEGER,
        Semana_Año INTEGER,
        Es_Fin_de_Semana INTEGER
    )
    """)
    
    # Borrar el contenido de la tabla si ya existe
    cursor.execute(f"DELETE FROM {table_name}")
    
    # Guardar el DataFrame en la tabla SQLite
    df.to_sql(table_name, conexion, if_exists='append', index=False)
    
    # Verificar que los datos se han guardado correctamente
    consulta = pd.read_sql(f'SELECT * FROM {table_name} LIMIT 5', conexion)
    
    # Cerrar la conexión
    conexion.close()

# Guardar la dimensión de tiempo en SQLite
guardar_dimension_tiempo_sqlite(df_dtiempo, nDWH, 'DimTiempo')

#### Cosechas

In [14]:
def clasificar_fecha(fecha, años):
    for año in años:
        if fecha >= pd.Timestamp(año, 1, 1) and fecha < pd.Timestamp(año, 7, 1):
            return f"{año} 1"
        elif fecha >= pd.Timestamp(año, 7, 1) and fecha < pd.Timestamp(año + 1, 1, 1):
            return f"{año} 2"
    return None

# Ejemplo de fechas y años para clasificación

# Definir el rango de fechas automáticamente
fecha_inicio = '2020-01-01'
fecha_fin = datetime(datetime.now().year, 12, 31)

fechas = pd.date_range(start=fecha_inicio, end=fecha_fin, freq='M')

fecha_inicio = datetime.strptime(fecha_inicio, '%Y-%m-%d')

# Crear una lista de años
años = list(range(fecha_inicio.year, fecha_fin.year + 1))


# Clasificar fechas en cosechas
cosechas = [clasificar_fecha(fecha, años) for fecha in fechas]

# Crear DataFrame con cosechas únicas
df_cosechas = pd.DataFrame(cosechas, columns=['cosecha']).drop_duplicates().reset_index(drop=True)

df_cosechas['id'] = df_cosechas.index + 1

# Conectar a SQLite y crear la tabla si no existe
conn = sqlite3.connect(nDWH)
cursor = conn.cursor()

# Crear tabla si no existe
create_table_query = """
CREATE TABLE IF NOT EXISTS dimCosechas (
    id INTEGER PRIMARY KEY,
    cosecha TEXT
)
"""
cursor.execute(create_table_query)

cursor.execute("""

delete from dimCosechas

""")


# Insertar datos en la tabla
insert_query = "INSERT INTO dimCosechas (id, cosecha) VALUES (?, ?)"
for index, row in df_cosechas.iterrows():
    cursor.execute(insert_query, (row['id'], row['cosecha']))

# Confirmar cambios
conn.commit()

# Verificar que los datos se hayan insertado correctamente
select_query = "SELECT * FROM dimCosechas"
result = pd.read_sql_query(select_query, conn)
print(result)

# Cerrar conexión
conn.close()

   id cosecha
0   1  2020 1
1   2  2020 2
2   3  2021 1
3   4  2021 2
4   5  2022 1
5   6  2022 2
6   7  2023 1
7   8  2023 2
8   9  2024 1
9  10  2024 2


#### Rodamientos

In [16]:
# Lista de clasificación mora
clasificacion_mora = ["Sin vencimiento", "Próximo a pasar a 1 día", "Próximo a pasar a 30 días", 
                      "Próximo a pasar a 30 días", "Próximo a pasar a 60 días", "Próximo a pasar a 60 días", 
                      "Próximo a pasar a 90 días", "Próximo a pasar a 90 días", "Próximo a pasar a 120 días", 
                      "Próximo a pasar a 120 días", "Próximo a pasar a 180 días", "Próximo a pasar a 180 días", 
                      "Más de 180 días"]

# Crear DataFrame
df_clasificacion_mora = pd.DataFrame(clasificacion_mora, columns=['clasificacion_mora'])

df_clasificacion_mora['id']=df_clasificacion_mora.index+1

# Conectar a SQLite y crear la tabla si no existe
conn = sqlite3.connect(nDWH)
cursor = conn.cursor()

# Crear tabla si no existe
create_table_query = """
CREATE TABLE IF NOT EXISTS dim_rodamientos (
    id INTEGER PRIMARY KEY,
    clasificacion_mora TEXT
)
"""
cursor.execute(create_table_query)


cursor.execute(""" 

delete from dim_rodamientos

""")


# Insertar datos en la tabla
insert_query = "INSERT INTO dim_rodamientos (id, clasificacion_mora) VALUES (?, ?)"
for index, row in df_clasificacion_mora.iterrows():
    cursor.execute(insert_query, (row['id'], row['clasificacion_mora']))

# Confirmar cambios
conn.commit()

# Verificar que los datos se hayan insertado correctamente
select_query = "SELECT * FROM dim_rodamientos"
result = pd.read_sql_query(select_query, conn)
print(result)

# Cerrar conexión
conn.close()

    id          clasificacion_mora
0    1             Sin vencimiento
1    2     Próximo a pasar a 1 día
2    3   Próximo a pasar a 30 días
3    4   Próximo a pasar a 30 días
4    5   Próximo a pasar a 60 días
5    6   Próximo a pasar a 60 días
6    7   Próximo a pasar a 90 días
7    8   Próximo a pasar a 90 días
8    9  Próximo a pasar a 120 días
9   10  Próximo a pasar a 120 días
10  11  Próximo a pasar a 180 días
11  12  Próximo a pasar a 180 días
12  13             Más de 180 días


#### Franjas de Mora

In [15]:
# Lista de franjas de mora
franjas_mora = ["Al día", "0 a 29 días", "30 a 59 días", "60 a 89 días", "90 a 119 días", "120 a 149 días", "150 a 179 días", "mayor a 180"]

# Crear DataFrame
df_franjas_mora = pd.DataFrame(franjas_mora, columns=['franja_mora'])

df_franjas_mora['id'] = df_franjas_mora.index+1

# Conectar a SQLite y crear la tabla si no existe
conn = sqlite3.connect(nDWH)
cursor = conn.cursor()

# Crear tabla si no existe
create_table_query = """
CREATE TABLE IF NOT EXISTS dim_franjas_mora (
    id INTEGER PRIMARY KEY,
    franja_mora TEXT
)
"""
cursor.execute(create_table_query)


cursor.execute("""
               
Delete from dim_franjas_mora
               
               """)


# Insertar datos en la tabla
insert_query = "INSERT INTO dim_franjas_mora (id, franja_mora) VALUES (?, ?)"
for index, row in df_franjas_mora.iterrows():
    cursor.execute(insert_query, (row['id'], row['franja_mora']))

# Confirmar cambios
conn.commit()

# Verificar que los datos se hayan insertado correctamente
select_query = "SELECT * FROM dim_franjas_mora"
result = pd.read_sql_query(select_query, conn)
print(result)

# Cerrar conexión
conn.close()

   id     franja_mora
0   1          Al día
1   2     0 a 29 días
2   3    30 a 59 días
3   4    60 a 89 días
4   5   90 a 119 días
5   6  120 a 149 días
6   7  150 a 179 días
7   8     mayor a 180


## Fact

### Extract

#### Main

In [17]:
### Query main

vsql = """		select	l.id
			,l.payment_reference
			,u.id as IdUniversidad
			,u.name as Universidad
			,c.description as Sede
			,l.created_at as Fecha_Solicitud 
			,l.state as IdEstado
			,ls.descripcion as Estado
			,case when u2.document_number = l.document_number then 'A nombre propio' else 'Por un tercero' end as self_requested
			,ap.name AS academic_program_name
			,u2.document_number AS user_document_number
			,l.document_number AS loan_taker_document_number
			,l.legalized_at
			,case when u2.origen is null then '' else u2.origen end as "origen"
			,case when u2.origen_otro is null then '' else u2.origen_otro end as "origen_otro"
			,l.installments as cuotas
			,((l.amount + ((case when json_extract_path_text("quote",'fee') is null then administrative_cost::text else json_extract_path_text("quote",'fee') end)::FLOAT)*(1-l.installments))/l.installments )::numeric::integer as capital_primercuota
			,case when json_extract_path_text("quote",'fee') is null then administrative_cost::text else json_extract_path_text("quote",'fee') end		"costo_serv"  
			,l.amount		"valor_solicitado" 
			,((l.amount + (case when json_extract_path_text("quote",'fee') is null then administrative_cost::text else json_extract_path_text("quote",'fee') end)::FLOAT)/l.installments)::numeric::integer	as primer_cuota
			,((l.amount + (case when json_extract_path_text("quote",'fee') is null then administrative_cost::text else json_extract_path_text("quote",'fee') end)::FLOAT))::numeric::integer as valor_apagar
			,l.semester AS semester
			,case when l.is_renovation is not null then (case when l.is_renovation is true then 'VERDADERO' else 'FALSO' end) else 'FALSO' end			"renovacion"
			,case when l.date_first_datacredictletter is not null then (to_char(l.date_first_datacredictletter , 'DD/MM/YYYY')) else '' end  as "primera_carta_datacredito"
			,case when l.date_second_datacredictletter is not null then (to_char(l.date_second_datacredictletter , 'DD/MM/YYYY')) else '' end   as "segunda_carta_datacredito"
		from loans l 
	inner join universities u on u.id = l.university_id
	left join campus c on c.id = l.campu_id
	inner join loans_states ls on ls.id = l.state
	inner join users u2 on l.user_id = u2.id
	inner join academic_programs ap on l.academic_program_id = ap.id
	where l.id not in (144,246,503,572,1025,1146,1315,1319,1400,1436,1488,1549,1580,1595,2068,177,1287,1339,1343,1732,1864,2039,2106,2335,96,4,9,24,45,46,48,58,59,65,66,68,
	75,76,124,143,146,147,148,149,152,154,155,156,159,167,170,178,183,193,196,204,207,209,210,212,218,219,221,222,227,231,242,244,261,270,298,308,310,318,323,331,341,346,350,
	354,368,372,378,393,395,400,404,406,425,427,430,433,435,437,443,452,458,464,467,469,470,471,475,476,477,479,480,481,489,491,493,501,504,506,507,508,509,513,517,518,520,528,
	529,538,544,546,547,548,549,554,557,566,579,588,589,607,608,609,611,612,614,617,620,632,637,654,655,660,662,663,666,668,677,680,684,690,692,695,709,714,719,724,734,744,745,748,
	749,751,754,762,774,782,790,791,793,801,802,804,805,806,815,817,819,827,833,835,841,842,845,846,853,854,871,875,876,877,887,889,891,896,901,903,905,908,913,919,924,927,930,942,
	945,953,960,963,965,969,970,982,991,992,997,1000,1002,1003,1011,1019,1020,1022,1026,1028,1029,1033,1041,1046,1048,1070,1072,1087,1091,1099,1111,1112,1114,1136,1144,1147,1154,1168,
	1173,1175,1178,1186,1188,1195,1205,1209,1213,1215,1216,1219,1223,1234,1236,1243,1248,1249,1251,1257,1278,1294,1295,1302,1303,1318,1338,1345,1349,1355,1363,1371,1375,1381,1383,1384,
	1388,1392,1393,1397,1398,1406,1408,1410,1411,1421,1441,1444,1447,1466,1476,1482,1483,1486,1490,1496,1502,1510,1520,1528,1533,1540,1544,1555,1557,1558,1581,1583,1584,1587,1601,1602,1603
	,1614,1616,1618,1621,1626,1627,1628,1653,1656,1658,1678,1680,1683,1689,1700,1707,1709,1710,1715,1716,1717,1718,1728,1731,1761,1765,1781,1782,1785,1797,1798,1809,1813,1816,1818,1826,1843
	,1852,1854,1855,1860,1878,1880,1881,1883,1895,1896,1898,1906,1923,1934,1936,1946,1963,1964,1965,1969,1973,1974,1978,1979,1980,1985,1992,1993,1996,1997,2001,2002,2003,2004,2005,2009,2013
	,2018,2025,2026,2028,2033,2038,2043,2044,2054,2059,2066,2067,2073,2078,2085,2086,2096,2097,2102,2117,2119,2121,2122,2130,2131,2133,2134,2138,2143,2147,2149,2151,2153,2158,2172,2173,2181,
	2184,2187,2191,2192,2193,2198,2201,2204,2213,2214,2227,2235,2239,2240,2244,2246,2256,2260,2266,2273,2275,2278,2280,2281,2282,2289,2294,2295,2297,2298,2306,2307,2315,2321,2324,2325,2336,2339
	,2340,2343,2356,2361,2371,2372,2382,2389,2395,2399,2400,2404,2407,2408,2414,2416,2423,2427,2429,2444,2450,2453,2458,2463,2466,2467,2468,2478,2480,2485,2487,2495,2497,2502,2503,2512,2513,2514,
	2515,2516,2518,2526,2527,2531,2534,2537,2538,2553,2567,2571,2581,2582,2588,2603,2608,2615,2620,2636,2643,2645,2648,2659,2661,2662,2665,2666,2672,2677,2681,2684,2685,2687,2696,2704,2706,2708,2710,
	2711,2712,2715,2716,2719,2720,2721,2722,2723,2725,2726,2729,2735,2738,2739,2741,2746,2748,2750,2755,2756,2757,2758,2759,2760,2767,2776,2801,2802,2803,2806,2807,2814,2815,2822,2824,2826,2828,2831,
	2832,2834,2843,2845,2846,2854,2855,2856,2860,2863,2865,2866,2868,2875,2879,2881,2885,2895,2902,2910,2914,2916,2918,2919,2921,2922,2925,2927,2930,2932,2942,2954,2958,2960,2961,2962,2972,2974,2975,2977,
	2978,2988,3001,3002,3004,3005,3006,3007,3009,3016,3017,3018,3019,3025,3026,3027,3028,3033,3036,3040,3043,3044,3048,3052,3063,3064,3065,3069,133,197,213,366,384,559,571,592,631,667,679,685,689,770,779,
	788,795,897,916,935,968,1009,1134,1139,1310,1757,1803,1966,2007,2016,2022,2083,2088,2189,2209,2220,2258,2271,2314,2316,2345,2432,2543,2595,2752,2857,2858,3045,5394) 
			
	
		
	
"""

In [18]:
### read sql main

try:
   
    engine = create_engine(strCnn)
    cnn = engine.connect()
    result = cnn.execute(text(vsql))
    
    df_main=pd.DataFrame(result.fetchall(), columns=result.keys())
  
    
except Exception as e:
    print("Ocurrío un error la intentar conectarse con el servidor de BD: ", e)

finally:
    cnn.close()

print('cantidad de registros {}'.format(df_main.shape[0]))


df_main.head()  

cantidad de registros 23924


,id,payment_reference,iduniversidad,universidad,sede,fecha_solicitud,idestado,estado,self_requested,academic_program_name,...,cuotas,capital_primercuota,costo_serv,valor_solicitado,primer_cuota,valor_apagar,semester,renovacion,primera_carta_datacredito,segunda_carta_datacredito
0,7040,715987698,7,Unir - Pregrado - Fundación Universitaria Inte...,None,2021-12-03 17:29:10.225209,90,Pagado,A nombre propio,Administración en Salud,...,6,111714,216728,1753922,328442,1970650,9,FALSO,,
1,7169,810761650,1,Areandina - Fundación Universitaria Del Área A...,Bogotá,2021-12-16 14:01:07.054082,90,Pagado,A nombre propio,Licenciatura en Educación Infantil - Presencia...,...,7,155025,339137,3120000,494162,3459137,7,FALSO,,10/04/2022
2,8940,386424160,17,UNINCCA - Universidad Incca De Colombia,None,2022-02-14 17:23:48.371186,90,Pagado,A nombre propio,Psicología,...,6,264643,329564,3235680,594207,3565244,1,FALSO,21/07/2022,
3,8122,439902626,1,Areandina - Fundación Universitaria Del Área A...,Bogotá,2022-01-18 17:26:13.697936,90,Pagado,Por un tercero,Optometría - Presencial PEREIRA,...,6,522259,590489,6086000,1112748,6676489,1,FALSO,,
4,8208,821055471,1,Areandina - Fundación Universitaria Del Área A...,Valledupar,2022-01-19 19:23:11.310032,90,Pagado,A nombre propio,Administración de Empresas - Virtual,...,6,99702,205158,1624000,304860,1829158,6,FALSO,21/06/2022,


#### Payments

In [19]:
# Lectura de datos

vsql = """ select l.id, l.user_id, l.document_number,p.processor_description, ls.description,l.state,p.id as id_pago, 
       p.date, p.amount
         from loans l inner join loans_states ls on l.state=ls.id inner join payments p on l.id = p.loan_id
          """

try:
    engine = create_engine(strCnn)
    cnn = engine.connect()
    result = cnn.execute(text(vsql))
    
    dfp=pd.DataFrame(result.fetchall(), columns=result.keys())
   
except Exception as e:
    print("Ocurrío un error la intentar conectarse con el servidor de BD: ", e)

finally:
    cnn.close()

print('cantidad de registros {}'.format(dfp.shape[0]))

dfp.head()

cantidad de registros 81938


,id,user_id,document_number,processor_description,description,state,id_pago,date,amount
0,3334,2930,1031151766,Deposito Efectivo en Oficina,PAYED,90,9202,2021-06-21,301404
1,3356,3088,1000834143,Deposito Efectivo en Oficina,PAYED,90,9205,2021-06-21,10000
2,3200,2987,79871129,Recaudo Daviplata PAGO CUOTA FINKY 278612731,OBSOLETE,70,9210,2021-06-20,250000
3,3200,2987,79871129,Recaudo Daviplata PAGO CUOTA FINKY 278612731,OBSOLETE,70,9211,2021-06-20,343316
4,4311,3777,79894377,Ajuste pago PAGO CUOTA FINKY,PAYED,90,9212,2021-06-21,1783540


In [20]:
# Convirtiendo datos a fechas
dfp.date = dfp.date.astype(str)
dfp.loc[dfp.date.str[0:4]=='0023','date']=None
dfp.loc[dfp.date.str[0:4]=='None','date']=""
dfp.date.fillna("",inplace=True)
dfp.date = pd.to_datetime(dfp.date)

#### Schedule

In [21]:
# Cruce de Tabla loan con tabla schedule payments
vsql = """ select l.id, l.user_id, l.document_number, l.amount,ls.description,l.state,p.id as pago_sche, 
       p.pay_date, p.paid_date, p.quota, p.paid_value,p.fee_number,p.fee,p.state state_sch,p.im
         from loans l inner join loans_states ls on l.state=ls.id inner join payment_schedule p on l.id = p.loan_id
          """
try:
   
    engine = create_engine(strCnn)
    cnn = engine.connect()
    result = cnn.execute(text(vsql))
    
    df_cr=pd.DataFrame(result.fetchall(), columns=result.keys())
  
except Exception as e:
    print("Ocurrío un error la intentar conectarse con el servidor de BD: ", e)

finally:
    cnn.close()

print('cantidad de registros {}'.format(df_cr.shape[0]))

df_cr.head()

cantidad de registros 107412


,id,user_id,document_number,amount,description,state,pago_sche,pay_date,paid_date,quota,paid_value,fee_number,fee,state_sch,im
0,9349,9915,1140365679,5000000,APPROVED,30,39009,2022-06-20,NaT,None,None,1.0,919706,1,None
1,9258,6720,30666343,3170000,DENIED,20,37013,2022-05-20,NaT,None,None,1.0,703304,1,None
2,9349,9915,1140365679,5000000,APPROVED,30,39010,2022-07-20,NaT,None,None,2.0,919706,1,None
3,9349,9915,1140365679,5000000,APPROVED,30,39011,2022-08-20,NaT,None,None,3.0,919706,1,None
4,9258,6720,30666343,3170000,DENIED,20,37014,2022-06-20,NaT,None,None,2.0,703304,1,None


In [22]:
# Ajustar Quote tipo de dato a flotante

df_cr.loc[df_cr.quota == '',"quota"]='0' # Reemplazar vacios por 0
df_cr.quota = df_cr.quota.fillna('0')  # reemplazar nulos por 0
df_cr.quota = df_cr.quota.str.replace("!","",regex=False)
df_cr.quota = df_cr.quota.astype(float) # convertir a tipo de dato flotante


# Ajustar Paid Date a float

df_cr.loc[df_cr.paid_value == '',"paid_value"]='0' # Reemplazar vacios por 0
df_cr.paid_value = df_cr.paid_value.fillna('0')  # reemplazar nulos por 0
df_cr.paid_value = df_cr.paid_value.str.replace("!","",regex=False)
df_cr.paid_value = df_cr.paid_value.astype(float) # convertir a tipo de dato flotante

# Ajustar Fee tipo de dato a flotante

df_cr.loc[df_cr.fee == '',"fee"]='0' # Reemplazar vacios por 0
df_cr.fee = df_cr.fee.fillna('0')  # reemplazar nulos por 0
df_cr.fee = df_cr.fee.str.replace("!","",regex=False)
df_cr.fee = df_cr.fee.astype(float) # convertir a tipo de dato flotante


df_cr.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 107412 entries, 0 to 107411
Data columns (total 15 columns):
 #   Column           Non-Null Count   Dtype         
---  ------           --------------   -----         
 0   id               107412 non-null  int64         
 1   user_id          107412 non-null  int64         
 2   document_number  107397 non-null  object        
 3   amount           107412 non-null  int64         
 4   description      107412 non-null  object        
 5   state            107412 non-null  int64         
 6   pago_sche        107412 non-null  int64         
 7   pay_date         107213 non-null  datetime64[ns]
 8   paid_date        60312 non-null   datetime64[ns]
 9   quota            107412 non-null  float64       
 10  paid_value       107412 non-null  float64       
 11  fee_number       107407 non-null  float64       
 12  fee              107412 non-null  float64       
 13  state_sch        107412 non-null  int64         
 14  im               695

### Transform

#### Días de Mora

In [23]:
pd.Timestamp.now().date()

datetime.date(2024, 5, 26)

In [24]:
df_cr.sort_values(by=['id','fee_number'],ascending=True,inplace=True) # Ordeno por los campos de id y consecutivo de cuota
col=['document_number','id','state','fee_number','state_sch','pay_date','paid_date'] # asigno a la variable col siguientes columnas para seleccionarlos
dfx = df_cr[df_cr.state_sch==3][col].groupby(['id']).first().reset_index() # Selecciono el primer registro de cada cuota en mora para tener la fecha de pago con el cual hacer la diferencia de la mora
dfx['current_date']=pd.Timestamp.now() # Creo un campo con la fecha de corte que es a febrero
dfx['dias_mora'] =(dfx.current_date - dfx.pay_date).dt.days # calculo los días de mora 
df_dmora = dfx
df_dmora

,id,document_number,state,fee_number,state_sch,pay_date,paid_date,current_date,dias_mora
0,233,1000773764,80,2.0,3,2020-07-05 00:00:00,NaT,2024-05-26 19:54:44.133545,1421
1,407,39093363,80,5.0,3,2020-10-20 00:00:00,NaT,2024-05-26 19:54:44.133545,1314
2,456,93295153,80,2.0,3,2020-07-20 00:00:00,NaT,2024-05-26 19:54:44.133545,1406
3,627,1007336298,80,3.0,3,2020-08-20 00:00:00,NaT,2024-05-26 19:54:44.133545,1375
4,869,1014264456,80,3.0,3,2020-09-05 00:00:00,NaT,2024-05-26 19:54:44.133545,1359
...,...,...,...,...,...,...,...,...,...
3728,24622,1094935397,80,2.0,3,2024-05-19 19:00:00,NaT,2024-05-26 19:54:44.133545,7
3729,24624,43750966,80,2.0,3,2024-05-19 19:00:00,NaT,2024-05-26 19:54:44.133545,7
3730,24634,1128478038,80,2.0,3,2024-05-20 00:00:00,NaT,2024-05-26 19:54:44.133545,6
3731,24638,1065815141,80,2.0,3,2024-05-19 19:00:00,2024-05-11 09:42:22,2024-05-26 19:54:44.133545,7


#### Cuotas en Mora & Valor en Mora

In [25]:
df_cvenc =  df_cr[df_cr.state_sch==3].groupby(['id']).agg({'id':'count','quota':'sum'}).rename(columns={'id':'cuotas_venc'}).reset_index()
df_cvenc.rename(columns={'quota':'valor_mora'},inplace=True)
df_cvenc.valor_mora = df_cvenc.valor_mora.round(0)
df_cvenc

,id,cuotas_venc,valor_mora
0,233,4,1343244.0
1,407,1,215800.0
2,456,4,3744020.0
3,627,3,1483046.0
4,869,3,703392.0
...,...,...,...
3728,24622,1,734095.0
3729,24624,1,1178822.0
3730,24634,1,707991.0
3731,24638,1,531306.0


#### Saldo a Pagar 

In [26]:
x = (df_cr.state_sch==1) | (df_cr.state_sch==3)
dfx = df_cr[x].pivot_table(index='id',aggfunc={'quota':'sum','fee':'sum'},columns='state_sch').reset_index()
dfx = dfx[[(   'id', ''),('fee',  1),('quota',  3)]]
dfx.columns= ['id','fee','quota']
dfx.fee.fillna(0,inplace=True)
dfx.quota.fillna(0,inplace=True)
dfx['saldo']=dfx[['fee','quota']].sum(axis=1)
df_saldo = dfx
df_saldo

,id,fee,quota,saldo
0,1,1999054.0,0.0,1999054.0
1,233,0.0,1343244.0,1343244.0
2,407,0.0,215800.0,215800.0
3,456,0.0,3744020.0,3744020.0
4,627,0.0,1483046.0,1483046.0
...,...,...,...,...
9713,24791,2348604.0,0.0,2348604.0
9714,24792,2245266.0,0.0,2245266.0
9715,24793,2245266.0,0.0,2245266.0
9716,24794,4005540.0,0.0,4005540.0


#### Valor real Pagado

In [27]:
x = (~dfp.processor_description.str.contains('Pago análisis de cupo',na=False)) & (~dfp.processor_description.str.contains('Pago estudio de crédito', na=False))  & (~dfp.processor_description.str.contains('Pago modificacion de cupo', na=False))

dfx = dfp[x].groupby(['id'],as_index=False).agg({'amount':'sum'}).rename(columns={'amount':'pay_amount'})
df_tpagos = dfx
df_tpagos

,id,pay_amount
0,1,500
1,13,1679322
2,60,4187975
3,82,1823822
4,107,1679055
...,...,...
14466,24779,391434
14467,24781,374211
14468,24783,1566493
14469,24784,374211


#### Fecha limite de pago

In [28]:
# Pago cuota programada se pasa a mes
df_cr['pay_month'] = df_cr.pay_date.dt.to_period('M').dt.to_timestamp()
dfx = df_cr[df_cr.pay_month=='2024-03-01'][['id','pay_date']].rename(columns={'pay_date':'Fecha_lim_pago'})
dfx = pd.merge(df_cr.groupby(['id'],as_index=False).agg({'pay_date':'max'}),dfx,on='id', how='left')
df_flim=dfx
df_flim

,id,pay_date,Fecha_lim_pago
0,1,2023-04-05,NaT
1,122,2022-05-20,NaT
2,233,2020-10-05,NaT
3,407,2020-10-20,NaT
4,456,2020-10-20,NaT
...,...,...,...
17179,24791,2024-10-20,NaT
17180,24792,2024-10-20,NaT
17181,24793,2024-10-20,NaT
17182,24794,2024-10-20,NaT


#### Fecha Terminación

In [29]:
#La fecha de terminación en caso de los creditos aprobados es el ultimo pago realizado

dfx = df_cr.groupby('id',as_index=False).agg({'pay_date':'max','paid_date':'max'})
dfx =pd.merge(dfx,dfp.groupby('id',as_index=False).agg({'date':'max'}),on='id',how='outer').fillna("")
dfx['date_end']=dfx[['pay_date', 'paid_date', 'date']].max(axis=1)
df_enddate = dfx 
df_enddate

,id,pay_date,paid_date,date,date_end
0,1,2023-04-05,NaT,2023-03-06,2023-04-05 00:00:00
1,122,2022-05-20,2022-06-02 13:17:07,2022-06-02,2022-06-02 13:17:07
2,233,2020-10-05,2020-06-04 00:00:00,2020-06-04,2020-10-05 00:00:00
3,407,2020-10-20,2021-06-18 00:00:00,2021-06-18,2021-06-18 00:00:00
4,456,2020-10-20,2020-06-25 00:00:00,2020-06-25,2020-10-20 00:00:00
...,...,...,...,...,...
20345,9039,NaT,NaT,2022-02-23,2022-02-23 00:00:00
20346,9040,NaT,NaT,2022-02-23,2022-02-23 00:00:00
20347,14592,NaT,NaT,2023-02-20,2023-02-20 00:00:00
20348,14609,NaT,NaT,2023-02-20,2023-02-20 00:00:00


#### Fecha ultimo pago Mes

In [30]:


x = (~dfp.processor_description.str.contains('Pago análisis de cupo',na=False)) & (~dfp.processor_description.str.contains('Pago estudio de crédito', na=False))  & (~dfp.processor_description.str.contains('Pago modificacion de cupo', na=False)) 


df_pmes = dfp[x].groupby(['id']).agg({'date':'max','amount':'sum'})
df_pmes =df_pmes.reset_index()
df_pmes.rename(columns={'date':'fecha_pmes'},inplace=True)
df_pmes

,id,fecha_pmes,amount
0,1,2020-10-05,500
1,13,2020-10-03,1679322
2,60,2020-10-05,4187975
3,82,2021-02-02,1823822
4,107,2020-10-30,1679055
...,...,...,...
14466,24779,2024-05-22,391434
14467,24781,2024-05-25,374211
14468,24783,2024-05-24,1566493
14469,24784,2024-05-23,374211


#### Cruzando Datos

In [31]:
# Cruzando días de mora

df_sl = pd.merge(df_main,df_dmora[['id','dias_mora']],on='id',how='left')
df_sl.dias_mora.fillna(0,inplace=True)

print('Días de mora {}'.format(df_sl.shape[0]))

# Cruzando valor y cuotas vencidad de mora

df_sl = pd.merge(df_sl,df_cvenc,on='id',how='left')

print('Cuotas de mora {}'.format(df_sl.shape[0]))


df_sl.cuotas_venc.fillna(0,inplace=True)
df_sl.valor_mora.fillna(0,inplace=True)

#Cruzando saldo

df_sl = pd.merge(df_sl,df_saldo[['id','saldo']],on='id',how='left')

df_sl.saldo.fillna(0, inplace=True)

print('Saldo {}'.format(df_sl.shape[0]))


# Cruzando valor real pagado

df_sl = pd.merge(df_sl,df_tpagos,on='id',how='left')

print('Valor real pagado {}'.format(df_sl.shape[0]))



# Fecha de terminación

df_sl = pd.merge(df_sl,df_enddate[['id','date_end']],on='id',how='left')
df_sl.date_end = df_sl.date_end.dt.strftime('%Y%m%d')

print('Fecha de terminación {}'.format(df_sl.shape[0]))


# Fecha pago mes

df_sl = pd.merge(df_sl,df_pmes[['id','fecha_pmes']],on='id',how='left')
df_sl.fecha_pmes = df_sl.fecha_pmes.dt.strftime('%Y%m%d')

print('Fecha pago mes {}'.format(df_sl.shape[0]))


# Fecha limite de pagos

df_sl = pd.merge(df_sl,df_flim[['id','Fecha_lim_pago']],on='id',how='left')
df_sl.Fecha_lim_pago = df_sl.Fecha_lim_pago.dt.strftime('%Y%m%d')

print('Fecha limite pago {}'.format(df_sl.shape[0]))


df_sl

Días de mora 23924
Cuotas de mora 23924
Saldo 23924
Valor real pagado 23924
Fecha de terminación 23924
Fecha pago mes 23924
Fecha limite pago 23945


,id,payment_reference,iduniversidad,universidad,sede,fecha_solicitud,idestado,estado,self_requested,academic_program_name,...,primera_carta_datacredito,segunda_carta_datacredito,dias_mora,cuotas_venc,valor_mora,saldo,pay_amount,date_end,fecha_pmes,Fecha_lim_pago
0,7040,715987698,7,Unir - Pregrado - Fundación Universitaria Inte...,None,2021-12-03 17:29:10.225209,90,Pagado,A nombre propio,Administración en Salud,...,,,0.0,0.0,0.0,0.0,1970646.0,20220705,20220623,NaN
1,7169,810761650,1,Areandina - Fundación Universitaria Del Área A...,Bogotá,2021-12-16 14:01:07.054082,90,Pagado,A nombre propio,Licenciatura en Educación Infantil - Presencia...,...,,10/04/2022,0.0,0.0,0.0,0.0,3460136.0,20220620,20220620,NaN
2,8940,386424160,17,UNINCCA - Universidad Incca De Colombia,None,2022-02-14 17:23:48.371186,90,Pagado,A nombre propio,Psicología,...,21/07/2022,,0.0,0.0,0.0,0.0,3565625.0,20220720,20220719,NaN
3,8122,439902626,1,Areandina - Fundación Universitaria Del Área A...,Bogotá,2022-01-18 17:26:13.697936,90,Pagado,Por un tercero,Optometría - Presencial PEREIRA,...,,,0.0,0.0,0.0,0.0,6677303.0,20220621,20220621,NaN
4,8208,821055471,1,Areandina - Fundación Universitaria Del Área A...,Valledupar,2022-01-19 19:23:11.310032,90,Pagado,A nombre propio,Administración de Empresas - Virtual,...,21/06/2022,,0.0,0.0,0.0,0.0,1835006.0,20220628,20220628,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
23940,8098,425549911,1,Areandina - Fundación Universitaria Del Área A...,Virtual,2022-01-17 20:37:51.567664,90,Pagado,A nombre propio,Administración de Empresas - Virtual,...,,,0.0,0.0,0.0,0.0,1916067.0,20220620,20220531,NaN
23941,11614,0046576590804,1,Areandina - Fundación Universitaria Del Área A...,Virtual,2022-10-03 20:24:19.000000,110,Cancelado,A nombre propio,Psicología - Virtual,...,,,568.0,1.0,199272.0,1076364.0,219274.0,20230305,20221003,NaN
23942,12530,0069028074981,1,Areandina - Fundación Universitaria Del Área A...,Virtual,2022-12-26 17:46:47.000000,90,Pagado,A nombre propio,Psicología - Virtual,...,,,0.0,0.0,0.0,0.0,2696627.0,20230502,20230502,NaN
23943,11613,0023940505480,1,Areandina - Fundación Universitaria Del Área A...,Virtual,2022-10-03 19:24:10.000000,70,Obsoleto,Por un tercero,Psicología - Virtual,...,,,0.0,0.0,0.0,1315644.0,NaN,20230306,NaN,NaN


#### Cosechas

In [ ]:
def clasificar_fecha(fecha, años):
    
    for año in años:
        if fecha >= pd.Timestamp(año, 1, 1) and fecha < pd.Timestamp(año, 7, 1):
            return f"{año} 1"
        elif fecha >= pd.Timestamp(año, 7, 1) and fecha < pd.Timestamp(año + 1, 1, 1):
            return f"{año} 2"
    return None



# Convertir la columna a datetime
df_sl['legalized_at'] = pd.to_datetime(df_sl['legalized_at'])

# Extraer solo la fecha
#df_sl['legalized_at'] = df_sl['legalized_at'].dt.date

# Obtener el rango de años disponibles en la columna 'Fecha de legalización'
años_disponibles = sorted(df_sl[df_sl.legalized_at.dt.year>=2020]['legalized_at'].dt.year.unique())

# Aplicamos la función a la columna 'Fecha de legalización' para crear una nueva columna
df_sl['Cosechas'] = df_sl['legalized_at'].apply(lambda fecha: clasificar_fecha(fecha, años_disponibles))

df_sl.Cosechas.fillna('No Disponible',inplace=True)

#### Rodamientos

In [ ]:
# Definimos los bins (intervalos) y las etiquetas correspondientes
bins = [-float('inf'), -10, 0, 20, 29, 50, 59, 80, 89, 110, 119, 170, 179, float('inf')]
labels = ["Sin vencimiento", "Próximo a pasar a 1 día", "Próximo a pasar a 30 días", "Próximo a pasar a 30 días",
          "Próximo a pasar a 60 días", "Próximo a pasar a 60 días", "Próximo a pasar a 90 días", 
          "Próximo a pasar a 90 días", "Próximo a pasar a 120 días", "Próximo a pasar a 120 días", 
          "Próximo a pasar a 180 días", "Próximo a pasar a 180 días", "Más de 180 días"]

# Utilizamos la función cut para crear la nueva columna 'Clasificación Mora'
df_sl['Rodamientos'] = pd.cut(df_sl['dias_mora'], bins=bins, labels=labels, right=True,ordered=False).astype(str)

# Rellenamos los valores NaN con "Fuera de rodamiento"
df_sl['Rodamientos'] = df_sl['Rodamientos'].fillna("Fuera de rodamiento")

#### Franjas de Mora

In [ ]:
# Crearemos los bins (intervalos) y las etiquetas correspondientes
bins = [-1, 0, 29, 59, 89, 119, 149, 179, float('inf')]
labels = ["Al día", "0 a 29 días", "30 a 59 días", "60 a 89 días", "90 a 119 días", "120 a 149 días", "150 a 179 días", "mayor a 180"]

# Utilizamos la función cut para crear la nueva columna 'Franja de Mora'
df_sl['Franja de Mora'] = pd.cut(df_sl['dias_mora'], bins=bins, labels=labels, right=True)


In [ ]:
df_sl

,id,payment_reference,iduniversidad,universidad,sede,fecha_solicitud,idestado,estado,self_requested,academic_program_name,...,cuotas_venc,valor_mora,saldo,pay_amount,date_end,fecha_pmes,Fecha_lim_pago,Cosechas,Rodamientos,Franja de Mora
0,7169,810761650,1,Areandina - Fundación Universitaria Del Área A...,Bogotá,2021-12-16 14:01:07.054082,90,Pagado,A nombre propio,Licenciatura en Educación Infantil - Presencia...,...,0.0,0.0,0.0,3460136.0,20220620,20220620,NaN,2021 2,Próximo a pasar a 1 día,Al día
1,7040,715987698,7,Unir - Pregrado - Fundación Universitaria Inte...,None,2021-12-03 17:29:10.225209,90,Pagado,A nombre propio,Administración en Salud,...,0.0,0.0,0.0,1970646.0,20220705,20220623,NaN,2022 1,Próximo a pasar a 1 día,Al día
2,8243,967140061,1,Areandina - Fundación Universitaria Del Área A...,Virtual,2022-01-20 12:20:49.938904,90,Pagado,A nombre propio,Administración de Empresas - Virtual,...,0.0,0.0,0.0,1829154.0,20220620,20220616,NaN,2022 1,Próximo a pasar a 1 día,Al día
3,6658,469217948,1,Areandina - Fundación Universitaria Del Área A...,Virtual,2021-09-10 19:06:14.793017,90,Pagado,Por un tercero,Esp. en Auditoria en Salud - Virtual,...,0.0,0.0,0.0,1979125.0,20220305,20220305,NaN,2021 2,Próximo a pasar a 1 día,Al día
4,24246,0029752284575,1,Areandina - Fundación Universitaria Del Área A...,Virtual,2024-03-04 09:23:48.000000,80,Aprobado Legalizado,A nombre propio,Psicología - Virtual,...,1.0,2036.0,273128.0,813275.0,20240605,20240510,20240304,2024 1,Próximo a pasar a 30 días,0 a 29 días
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
23936,21462,0095230896424,1,Areandina - Fundación Universitaria Del Área A...,Virtual,2023-12-23 07:54:49.000000,90,Pagado,A nombre propio,Psicología - Virtual,...,0.0,0.0,0.0,3315824.0,20240525,20240525,20240320,2023 2,Próximo a pasar a 1 día,Al día
23937,23131,0071117533774,1,Areandina - Fundación Universitaria Del Área A...,Valledupar,2024-01-26 10:25:53.000000,80,Aprobado Legalizado,Por un tercero,Profesional en Entrenamiento Deportivo - Prese...,...,0.0,0.0,899319.0,4501508.0,20240620,20240521,20240320,2024 1,Próximo a pasar a 1 día,Al día
23938,6297,None,1,Areandina - Fundación Universitaria Del Área A...,Virtual,2021-08-05 00:22:18.313569,70,Obsoleto,Por un tercero,Psicología - Virtual,...,0.0,0.0,0.0,NaN,20240126,NaN,NaN,No Disponible,Próximo a pasar a 1 día,Al día
23939,23177,0026664164988,25,Salesiana - Fundación Universitaria Salesiana,None,2024-01-26 17:16:56.000000,80,Aprobado Legalizado,A nombre propio,Química Farmaceutica - Presencial,...,0.0,0.0,1036009.0,5180045.0,20240620,20240506,20240320,2024 1,Próximo a pasar a 1 día,Al día


### LOAD

In [ ]:
# Crear una conexión a la base de datos SQLite
engine = create_engine('sqlite:///dwh_finky.db')

# Insertar el DataFrame en una tabla llamada 'mi_tabla'
df_sl.to_sql('dwh_sl', con=engine, if_exists='replace', index=False)

69897

In [133]:
conn = sqlite3.connect('finky.db')
c = conn.cursor()

# Ejecutamos la instrucción DELETE
c.execute('''
    DROP TABLE dimUniversidad
''')

conn.commit()
conn.close()

OperationalError: no such table: dimUniversidad

In [139]:
conn = sqlite3.connect(nDWH)
c = conn.cursor()

# Ejecutamos la instrucción DELETE
c.execute('''
    DROP TABLE dimUniversidad
''')

conn.commit()
conn.close()

OperationalError: database is locked

In [138]:
conn.commit()
conn.close()

In [33]:
conn = sqlite3.connect('dwh_finky.db')
c = conn.cursor()

# Convertir fechas a formato de texto compatible con SQLite
df_sl['fecha_solicitud'] = df_sl['fecha_solicitud'].dt.strftime('%Y-%m-%d')
#df_sl['legalized_at'] = df_sl['legalized_at'].dt.strftime('%Y-%m-%d')
#df_sl['date_end'] = df_sl['date_end'].dt.strftime('%Y-%m-%d')
#df_sl['fecha_pmes'] = df_sl['fecha_pmes'].dt.strftime('%Y-%m-%d')
#df_sl['Fecha_lim_pago'] = df_sl['Fecha_lim_pago'].dt.strftime('%Y-%m-%d')



c.execute('''
    CREATE TABLE IF NOT EXISTS dwh_sl2 (
        id INTEGER                                                                                                                                                                                   KEY,
        payment_reference TEXT,
        iduniversidad INTEGER,
        universidad TEXT,
        sede TEXT,
        fecha_solicitud TEXT,
        idestado INTEGER,
        estado TEXT,
        self_requested TEXT,
        academic_program_name TEXT,
        user_document_number TEXT,
        loan_taker_document_number TEXT,
        legalized_at TEXT,
        origen TEXT,
        origen_otro TEXT,
        cuotas INTEGER,
        capital_primercuota INTEGER,
        costo_serv TEXT,
        valor_solicitado INTEGER,
        primer_cuota INTEGER,
        valor_apagar INTEGER,
        semester INTEGER,
        renovacion TEXT,
        primera_carta_datacredito TEXT,
        segunda_carta_datacredito TEXT,
        dias_mora REAL,
        cuotas_venc REAL,
        valor_mora REAL,
        saldo REAL,
        pay_amount REAL,
        date_end TEXT,
        fecha_pmes TEXT,
        Fecha_lim_pago TEXT
    )
''')

# Insertamos los registros en la tabla
for index, row in df_sl.iterrows():
    c.execute('''
        INSERT INTO dwh_sl2 (
            id, payment_reference, iduniversidad, universidad, sede, fecha_solicitud, idestado, estado,
            self_requested, academic_program_name, user_document_number, loan_taker_document_number, legalized_at,
            origen, origen_otro, cuotas, capital_primercuota, costo_serv, valor_solicitado, primer_cuota, valor_apagar,
            semester, renovacion, primera_carta_datacredito, segunda_carta_datacredito, dias_mora, cuotas_venc,
            valor_mora, saldo, pay_amount, date_end, fecha_pmes, Fecha_lim_pago
        ) VALUES (?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?)
    ''', tuple(row))

# Guardamos los cambios y cerramos la conexión
conn.commit()
conn.close()